In [4]:
#Import necessary libraries
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, births_in_year, births_per_year
import altair as alt

#Disable max rows in Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Load Data

In [5]:
dfraw = load_names_from_web(category='national', hide_pre_1937=False, use_existing_files=True)

#find first and last known year
first_known_year = dfraw['year'].min()
last_known_year = dfraw['year'].max()

# function to get births per year
births_by_year = births_per_year()

# dfraw = dfraw[dfraw['name'] == 'Sarah']

#copy data
all_years = dfraw.copy()

In [ ]:
# create empyt dataframe to store data
data = pd.DataFrame()

for year in range(first_known_year, last_known_year+1):

    print(year) #display current year
    
    #copy data for current year
    df = all_years.copy()
    # calculate cum sum and median age
    df = df[df['year'] <= year]
    df = df.sort_values(by='year')
    df['cumsum'] = df.groupby(['name', 'M/F'])['count'].cumsum()
    df['sum'] = df.groupby(['name', 'M/F'])['count'].transform('sum')
    medians = df[df['cumsum'] >= df['sum']/2]
    medians = medians.drop_duplicates(subset=['name', 'M/F'], keep='first')
    medians['age'] = year - medians['year']
    # calculate coutn for current year; merge with median data
    thisyear = df[df['year'] == year][['name', 'M/F', 'count']].rename(columns={'count': 'thisyear_count'})
    df = medians.merge(thisyear, how='left', on=['name', 'M/F'])
    df['thisyear_count'] = df['thisyear_count'].fillna(0)
    df['thisyear_count_adj'] = df['thisyear_count'].replace(0, 2)
    df['thisyear_rank'] = df.groupby('M/F')['thisyear_count'].rank(method='first', ascending=False)
    df['thisyear_pct'] = df.apply(lambda row: row['thisyear_count'] / births_by_year.at[year, row['M/F']], axis=1)
    df['thisyear_pct_adj'] = df.apply(lambda row: row['thisyear_count_adj'] / births_by_year.at[year, row['M/F']], axis=1)
    df = df[['state', 'name', 'M/F', 'sum', 'age', 'thisyear_count', 'thisyear_rank', 'thisyear_pct', 'thisyear_count_adj', 'thisyear_pct_adj']]
    df['year'] = year
    # concatenate processed data
    data = pd.concat([data, df])

# display dataframe 
data

1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985


In [ ]:
#store backup data
backup = data.copy()

In [ ]:
#filter data
data = backup.copy()
data = data[data['thisyear_rank'] <= 100]
data = data[data['year'] % 5 == 0]
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']

#Create Altair charts for male/female data
chart_F = alt.Chart(data_F).mark_line().encode(
    x='thisyear_rank',
    y='thisyear_pct',
    tooltip=['name', 'M/F', 'year', 'age', 'thisyear_count', 'sum'],
    color='year'
).properties(
    width=450,
    height=400
)
chart_M = alt.Chart(data_M).mark_line().encode(
    x='thisyear_rank',
    y='thisyear_pct',
    tooltip=['name', 'M/F', 'year', 'age', 'thisyear_count', 'sum'],
    color=alt.Color('year', scale=alt.Scale(scheme='greys'))
).properties(
    width=450,
    height=400
)

#display charts
chart_F | chart_M

# ranks need to be cleaned up by gender

In [ ]:
data = backup.copy()
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']

#function to calcualte top shares
def top_shares(df):
    results = []
    groups = df.groupby('year')
    for year, group in groups:
        group = group.sort_values(by='thisyear_rank')
        for x in [1, 10, 100, 1000]:
            top_x = group.head(x)
            total_pct = top_x['thisyear_pct'].sum()
            results.append({'year': year, 'top_x': x, 'total_pct': total_pct})
    return pd.DataFrame(results)

#calculate top shared for male/female
data_F = top_shares(data_F)
data_M = top_shares(data_M)

#create Altair charts for top shares
chart_F = alt.Chart(data_F).mark_line().encode(
    x='year',
    y='total_pct',
    tooltip=['year', 'top_x', 'total_pct'],
    color=alt.Color('top_x', scale=alt.Scale(scheme='greys', reverse=True, type='log'))
).properties(
    width=450,
    height=400
)
chart_M = alt.Chart(data_M).mark_line().encode(
    x='year',
    y='total_pct',
    tooltip=['year', 'top_x', 'total_pct'],
    color='top_x'
).properties(
    width=450,
    height=400
)

#display charts
chart_F | chart_M

In [ ]:
## Calculate top shares between male/female
data_diff = data_F.merge(data_M, how='inner', on=['year', 'top_x'], suffixes=('_f', '_m'))
data_diff['diff'] = data_diff['total_pct_f'] - data_diff['total_pct_m']

chart_diff = alt.Chart(data_diff).mark_line().encode(
    x='year',
    y='diff',
    tooltip=['year', 'top_x', 'diff'],
    color=alt.Color('top_x', scale=alt.Scale(scheme='greys', reverse=True, type='log'))
).properties(
    width=450,
    height=400
)

#display charts
chart_diff

In [ ]:
#Calculating Shannon entropy for male/female
data = backup.copy()
data = data[data['thisyear_rank'] <= 100000]
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']

#function to calculate Shannon entropy
def shannon_entropy(pcts):
    pcts = pcts[pcts > 0]
    return -np.sum(pcts * np.log2(pcts))

entropy_F = data_F.groupby('year')['thisyear_pct'].apply(shannon_entropy).reset_index().rename(columns={'thisyear_pct': 'entropy'})
entropy_M = data_M.groupby('year')['thisyear_pct'].apply(shannon_entropy).reset_index().rename(columns={'thisyear_pct': 'entropy'})
entropy_diff = pd.merge(entropy_F, entropy_M, how='inner', on='year', suffixes=('_f', '_m'))
entropy_diff['diff'] = entropy_diff['entropy_m'] - entropy_diff['entropy_f']

chart_F = alt.Chart(entropy_F).mark_line().encode(
    x='year',
    y='entropy',
    tooltip=['year', 'entropy'],
    color=alt.value('red')
).properties(
    width=450,
    height=400
)
chart_M = alt.Chart(entropy_M).mark_line().encode(
    x='year',
    y='entropy',
    tooltip=['year', 'entropy'],
    color=alt.value('blue')
).properties(
    width=450,
    height=400
)

#display charts
(chart_F | chart_M).resolve_scale(y='shared')

In [ ]:
chart_M = alt.Chart(entropy_diff).mark_line().encode(
    x='year',
    y='diff',
    tooltip=['year', 'diff'],
    color=alt.value('grey')
).properties(
    width=450,
    height=400
)

#display chart
chart_M